In [36]:
%load_ext lab_black
import pandas as pd
import json


f = open("histohour_BTC.json",)
data = json.load(f)
data = pd.DataFrame.from_dict(data["Data"]["Data"])
data = data[["time", "close"]]
f.close()
data

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


,time,close
0,1611691200,32020.72
1,1611694800,32024.18
2,1611698400,32765.79
3,1611702000,32516.52
4,1611705600,32371.54
...,...,...
1996,1618876800,54710.74
1997,1618880400,54854.55
1998,1618884000,54235.44
1999,1618887600,54602.38


In [19]:
def get_profit(lower_limit, upper_limit):
    i = 0
    profit = 0
    records = []
    n = 0
    for index, row in data.iterrows():
        day = index
        price = row["close"]
        if i % 2 == 0:
            if row["RSI"] <= lower_limit and n == 0:
                profit -= price
                records.append(["Buying", -round(price), round(profit), day])
                n += 1
                i += 1
        else:
            if row["RSI"] >= upper_limit and n >= 0 and profit + price >= 1:
                profit += price
                records.append(["Selling", round(price), round(profit), day])
                n -= 1
                i += 1
    return profit


def calculation(data, window_length, lower_limit, upper_limit):
    delta = data["close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up <= 1] = 0
    down[down >= 2] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS1 = roll_up1 / roll_down1
    RSI1 = 100.0 - (100.0 / (1.0 + RS1))
    data.loc[:, "RSI"] = RSI1
    data.loc[:, "TimeStamp"] = pd.to_datetime(data["time"], unit="s")
    data = data.set_index("TimeStamp")
    return get_profit(lower_limit, upper_limit)


calculation(data, 14, 50, 60)

-49882.219999999994

In [20]:
%%time
from tqdm import tqdm
optimizations=[]
for window_length in tqdm(range(10, 20)):
    for lower_limit in range(10, 30):
        for upper_limit in range(80, 100):
            profit = calculation(data, window_length, lower_limit,upper_limit)
            optimizations.append([profit, window_length, lower_limit, upper_limit])

100%|██████████| 10/10 [29:53<00:00, 179.33s/it]

Wall time: 29min 53s


In [22]:
optimization_df = pd.DataFrame(
    optimizations, columns=["Profit", "WindowLength", "LowerLimit", "UpperLimit"]
)
pd.set_option("max_rows", 30)
optimization_df

,Profit,WindowLength,LowerLimit,UpperLimit
0,-53430.10,10,10,80
1,-53430.10,10,10,81
2,-53430.10,10,10,82
3,-50925.31,10,10,83
4,-49825.54,10,10,84
...,...,...,...,...
3995,-30682.82,19,29,95
3996,-30682.82,19,29,96
3997,-30682.82,19,29,97
3998,-30682.82,19,29,98


In [27]:
optimization_df[optimization_df["Profit"] == optimization_df["Profit"].max()]

,Profit,WindowLength,LowerLimit,UpperLimit
1032,-17711.08,12,21,92


In [28]:
window_length = 12
lower_limit = 21
upper_limit = 92

In [30]:
def get_profit(lower_limit, upper_limit):
    i = 0
    profit = 0
    records = []
    n = 0
    for index, row in data.iterrows():
        day = index
        price = row["close"]
        if i % 2 == 0:
            if row["RSI"] <= lower_limit and n == 0:
                profit -= price
                records.append(["Buying", -round(price), round(profit), day])
                n += 1
                i += 1
        else:
            if row["RSI"] >= upper_limit and n >= 0 and profit + price >= 1:
                profit += price
                records.append(["Selling", round(price), round(profit), day])
                n -= 1
                i += 1
    return records, profit


def calculation(data, window_length, lower_limit, upper_limit):
    delta = data["close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up <= 1] = 0
    down[down >= 2] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS1 = roll_up1 / roll_down1
    RSI1 = 100.0 - (100.0 / (1.0 + RS1))
    data.loc[:, "RSI"] = RSI1
    data.loc[:, "TimeStamp"] = pd.to_datetime(data["time"], unit="s")
    data = data.set_index("TimeStamp")
    records, profit = get_profit(lower_limit, upper_limit)
    return records, profit


records, profit = calculation(data, window_length, lower_limit, upper_limit)
formated_profit = f"{round(profit):,}"
print(f"Total profit: USD {formated_profit}\n")
records_df = pd.DataFrame(records, columns=["Action", "Price", "Profit", "Day"])
pd.set_option("max_rows", 30)
records_df

Total profit: USD -17,711



,Action,Price,Profit,Day
0,Buying,-30316,-30316,17
1,Selling,43234,12918,304
2,Buying,-45010,-32092,354
3,Selling,54781,22688,573
4,Buying,-53756,-31068,640
5,Selling,52565,21498,856
6,Buying,-47354,-25857,929
7,Selling,61611,35754,1104
8,Buying,-55855,-20101,1141
9,Selling,60983,40882,1760


In [23]:
# optimization_df.to_excel("Profit_Optimization_BTC_hr.xlsx")

In [31]:
data

,time,close,RSI,TimeStamp
0,1611691200,32020.72,NaN,2021-01-26 20:00:00
1,1611694800,32024.18,100.000000,2021-01-26 21:00:00
2,1611698400,32765.79,100.000000,2021-01-26 22:00:00
3,1611702000,32516.52,71.650135,2021-01-26 23:00:00
4,1611705600,32371.54,59.964913,2021-01-27 00:00:00
...,...,...,...,...
1996,1618876800,54710.74,28.102360,2021-04-20 00:00:00
1997,1618880400,54854.55,32.076905,2021-04-20 01:00:00
1998,1618884000,54235.44,25.035511,2021-04-20 02:00:00
1999,1618887600,54602.38,35.025939,2021-04-20 03:00:00
